In [1]:
# Importamos las librerías necesarias
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, IntSlider
from scipy.integrate import odeint
import random

# Función de Gillespie para la simulación estocástica
def gillespie_gfp(IC, c1, c2, n):
    """
    Simula el proceso de producción y degradación de GFP usando el algoritmo de Gillespie.
    
    Args:
    - IC: Condición inicial (número inicial de moléculas de GFP)
    - c1: Tasa de producción de GFP
    - c2: Tasa de degradación de GFP
    - n: Número de pasos de tiempo
    
    Returns:
    - tvec: Tiempos en los que ocurren los eventos
    - xvec: Valores de GFP en cada paso de tiempo
    """
    # Inicializamos las variables
    t = 0
    x = IC
    tvec = [0]
    xvec = [x]
    
    for i in range(n):
        # Calculamos las tasas de los eventos (hazard functions)
        rate_production = c1
        rate_degradation = c2 * x
        
        # Suma de las tasas
        total_rate = rate_production + rate_degradation
        
        # Tiempo hasta el próximo evento
        dt = np.random.exponential(1 / total_rate)
        t += dt
        
        # Determinamos qué evento ocurre (producción o degradación)
        if random.random() < rate_production / total_rate:
            x += 1  # Producción de GFP
        else:
            x -= 1 if x > 0 else 0  # Degradación de GFP (no puede ser negativo)
        
        # Guardamos los resultados
        tvec.append(t)
        xvec.append(x)
    
    return np.array(tvec), np.array(xvec)

# Función para simular varias realizaciones
def simulate_multiple(IC, c1, c2, n, iterations, num_points=50):
    """
    Simula varias realizaciones del proceso estocástico de GFP.
    
    Args:
    - IC: Condición inicial
    - c1: Tasa de producción
    - c2: Tasa de degradación
    - n: Número de pasos de tiempo
    - iterations: Número de iteraciones (realizaciones)
    - num_points: Número de puntos en los que discretizar el tiempo
    
    Returns:
    - regular_out_matrix: Matriz con los resultados de todas las realizaciones
    - t_disc: Vector de tiempos discretizados
    """
    regular_out_matrix = np.zeros((num_points, iterations))
    x_end_vec = np.zeros(iterations)
    
    # Realizamos varias iteraciones
    for ii in range(iterations):
        t_vec, x_vec = gillespie_gfp(IC, c1, c2, n)
        
        # Discretizamos el tiempo para obtener una rejilla uniforme
        t_disc = np.linspace(0, max(t_vec), num_points)
        x_disc = np.interp(t_disc, t_vec, x_vec)
        
        regular_out_matrix[:, ii] = x_disc
        x_end_vec[ii] = x_vec[-1]  # Guardamos el valor final de GFP
    
    return regular_out_matrix, t_disc, x_end_vec

# Función para el modelo determinista
def det_gfp_model(y, t, c1, c2):
    """
    Ecuación diferencial para el modelo determinista de GFP.
    
    Args:
    - y: Valor actual de GFP
    - t: Tiempo
    - c1: Tasa de producción
    - c2: Tasa de degradación
    
    Returns:
    - Derivada de GFP respecto al tiempo
    """
    return c1 - c2 * y

# Función para correr la simulación y mostrar gráficos
def run_simulation(IC, c1, c2, n, iterations):
    """
    Corre la simulación estocástica y determinista y genera los gráficos correspondientes.
    
    Args:
    - IC: Condición inicial
    - c1: Tasa de producción
    - c2: Tasa de degradación
    - n: Número de pasos de tiempo
    - iterations: Número de iteraciones
    """
    # Simulación estocástica (varias realizaciones)
    regular_out_matrix, t_disc, x_end_vec = simulate_multiple(IC, c1, c2, n, iterations)
    
    # Gráfico de las realizaciones estocásticas y la media
    plt.figure(figsize=(10, 6))
    for i in range(iterations):
        plt.plot(t_disc, regular_out_matrix[:, i], color='gray', alpha=0.5)
    
    plt.plot(t_disc, np.mean(regular_out_matrix, axis=1), color='blue', lw=2, label='Media Estocástica')
    
    # Simulación determinista
    y0 = IC
    t = t_disc
    sol = odeint(det_gfp_model, y0, t, args=(c1, c2))
    
    plt.plot(t, sol, color='red', lw=2, label='Solución Determinista')
    
    plt.xlabel('Tiempo')
    plt.ylabel('GFP')
    plt.title('Simulación Estocástica vs Determinista de GFP')
    plt.legend()
    plt.grid(True)
    plt.show()
    
    # Histograma de los valores finales
    plt.figure(figsize=(8, 5))
    plt.hist(x_end_vec, bins=20, density=True, alpha=0.6, color='g')
    plt.axvline(x=c1 / c2, color='blue', lw=2, label=f'Valor Determinista: {c1 / c2}')
    plt.xlabel('Valor Final de GFP')
    plt.ylabel('Densidad')
    plt.title('Distribución de los Valores Finales de GFP')
    plt.legend()
    plt.grid(True)
    plt.show()

# Función interactiva con sliders
interact(run_simulation,
         IC=IntSlider(value=1, min=0, max=50, step=1, description='IC (GFP Inicial)'),
         c1=FloatSlider(value=100, min=1, max=200, step=1, description='Tasa de Producción (c1)'),
         c2=FloatSlider(value=5, min=0.1, max=10, step=0.1, description='Tasa de Degradación (c2)'),
         n=IntSlider(value=100, min=10, max=500, step=10, description='Pasos de Tiempo (n)'),
         iterations=IntSlider(value=100, min=10, max=500, step=10, description='Iteraciones'));


interactive(children=(IntSlider(value=1, description='IC (GFP Inicial)', max=50), FloatSlider(value=100.0, des…